## Computational Chemistry for Experimentalists
## Chapter 7: The Art of Interpretation II: Partial Charges, Oxidation States, and Bond Orders 

Partial atomic charges, oxidation states of a metal in a complex, and chemical bond orders are core chemical concepts that prove surprisingly slippery in practice. There is no single, rigorous, universally acceptable definition of the bond order of a covalent bond, or the charge on an atom in a molecule, or the oxidation state of an atom in a complex. This set of examples illustrate how different definitions of these quantities all can build chemical insight.

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import math
import numpy
import matplotlib.pyplot as plt
from pyscf import gto,scf,dft,cc
from pyscf.tools import cubegen
import py3Dmol
from pyscf.tools import molden 

## Example 1: Water Partial Charges  

The electric dipole moment of an isolated water molecule is an unambiguous observable. The partial atomic charges are not. While the hydrogens should have charge +delta and the oxygen should have charge -2 delta, there's no single "right" answer for delta. 

This first block generates the geometry and HF/3-21G wavefunction of a single water molecule. 

In [ ]:
m=Chem.MolFromSmiles('O')
m2=Chem.AddHs(m)
AllChem.EmbedMolecule(m2)
AllChem.MMFFOptimizeMolecule(m2)
m3=Chem.MolToMolBlock(m2)
elements = [atom.GetSymbol() for atom in m2.GetAtoms()]
coordinates = m2.GetConformer().GetPositions()
atoms = [(element, coordinate) for element, coordinate in zip(elements, coordinates)]
m4 = gto.Mole(basis="cc-pvtz")
m4.atom = atoms
m4.build();
mf=scf.RHF(m4)
mf.kernel() 
p = py3Dmol.view(width=200,height=200)
p.addModel(m3, 'sdf')
p.setStyle({'stick':{},'sphere':{"scale":0.1}})
p.render()

This block computes the dipole moment and two choices of partial charge: 

In [ ]:
d=mf.dip_moment()
dmag=numpy.dot(d,d)**0.5
deltaMulliken=mf.mulliken_pop()
deltaLowdin=mf.mulliken_pop_meta_lowdin_ao()
print(deltaLowdin[1])

print('Dipole moment in debye: %.3f'%(dmag))
print('H atom charge delta')
print('%20s  %.3f'%('Mulliken',deltaMulliken[1][1]))
print('%20s  %.3f'%('Lowdin',deltaLowdin[1][1]))

This block writes the PySCF wavefunction as a .molden file for post-processing by Multiwfn. The Multiwfn package can compute multiple other definitions of partial charge. Here's a brief summary of the computed hydrogen atom partial charges in H2O. Note that the Mulliken charge matches PySCF and that the Lowdin charge is not correct. The charges based on bond order (Hirshfeld, Mulliken) are different from the charges based on electrostatics (Becke, RESP, ADCH, CHELP) :

|Method |H atom charge|
| -- | -- |
|Hirshfeld|  0.162287|
|VDD| 0.145298|
|Mulliken|  0.24419|
|Lowdin|  -0.16996|
|Becke|  0.356394 |
|ADCH| 0.356402  |
|CHELP| 0.364839||
|Merz-Kollman| 0.361579|
|CMI| 0.325707 |
|RESP|  0.361611|

In [ ]:
with open('H2O.molden', 'w') as f1:
    molden.header(m4, f1)
    molden.orbital_coeff(m4, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

## Example 2: Oxidation States 

The formal oxidation state of a metal atom in a complex doesn't match any known definition of partial charge. Here we compute the Mulliken and Lowden charges on the formally Fe(III) metal atom of high-spin FeCl6(3-). 

In [ ]:
m=gto.Mole(atom='Fe 0.0 0.0 0.0; Cl 2.55 0.0 0.0;Cl -2.55 0.0 0.0;Cl 0.0 2.55 0.0; Cl 0.0 -2.55 0.0; Cl 0.0 0.0 2.55; Cl 0.0 0.0 -2.55',basis='6-31g',charge=-3,spin=5)
m.build()
mf=scf.UHF(m)
mf.kernel()
deltaMulliken=mf.mulliken_pop()
deltaLowdin=mf.mulliken_pop_meta_lowdin_ao()
#print(deltaMulliken[1])
#print(deltaLowdin[1])
print('Fe atom charge')
print('%20s  %.3f'%('Mulliken',deltaMulliken[1][0]))
print('%20s  %.3f'%('Lowdin',deltaLowdin[1][0]))

## Example 3: Bond Orders 
This example computes the H-H bond order of H2 at two different bond lengths. The bond order "should" be 1 near equilibrium and 0 at dissociation, but what happens when this is computed explicitly?

This first block computes the RHF/3-21G orbitals of H2 near equilibrium and for a stretched bond, and writes both as Molden files. 

In [ ]:
m=gto.Mole(atom='H 0.0 0.0 0.0; H 0.0 0.0 0.74',basis='3-21g')
m.build()
mf=scf.RHF(m)
mf.kernel()
with open('H2eq.molden', 'w') as f1:
    molden.header(m, f1)
    molden.orbital_coeff(m, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)
m=gto.Mole(atom='H 0.0 0.0 0.0; H 0.0 0.0 2.0',basis='3-21g')
m.build()
mf=scf.RHF(m)
mf.kernel()
tw') as f1:
    molden.header(m, f1)
    molden.orbital_coeff(m, f1, mf.mo_coeff, ene=mf.mo_energy, occ=mf.mo_occ)

## Results 

| Method | Equilibrium | Stretched |
| -- |--|--|
| Mayer |1.00000000|1.00000000|
| Wiberg |1.00000000|1.00000000|
| Mulliken |0.80825721|0.33344113|
| FBO |0.99999991|0.99999659|
| LBO |0.737313|<0.05|


# Practicel Problems: 
Part 1: Compute the partial atomic charges of the H atom in methane, and compare the results to the partial atomic charge of H atom in water. You should find that the chemical trends are sensible (i.e., the H atom in water is more positively charged), even if the values are differnt.

Part 2: Compute the partial charge of iron atom in high-spin iron(II) and iron(III) hexa-aquo complexes. You can use the following geometry for both 

                        Standard orientation:
 ---------------------------------------------------------------------
 Center     Atomic      Atomic             Coordinates (Angstroms)
 Number     Number       Type             X           Y           Z
 ---------------------------------------------------------------------
      1         26           0        0.000218    0.000026   -0.000076
      2          8           0        1.077455    1.729495   -0.205007
      3          1           0        0.857422    2.590219    0.224658
      4          8           0       -1.142871    0.520039   -1.617825
      5          1           0       -1.035324    1.345604   -2.147665
      6          8           0        1.314169   -0.966074   -1.238331
      7          1           0        1.229139   -1.028834   -2.219510
      8          8           0       -1.314543    0.965193    1.238319
      9          1           0       -2.130759    1.435726    0.944618
     10          8           0        1.142649   -0.519670    1.617671
     11          1           0        1.883637    0.017974    1.986101
     12          8           0       -1.077345   -1.729167    0.205280
     13          1           0       -1.892143   -1.829129    0.752952
     14          1           0        2.128900   -1.439136   -0.944574
     15          1           0       -0.860345   -2.589067   -0.227544
     16          1           0        1.035752   -1.345198    2.147691
     17          1           0       -1.228386    1.030324    2.219230
     18          1           0        1.894896    1.829084   -0.748831
     19          1           0       -1.884579   -0.016782   -1.985992

Part 3: Compute the P-H and P=O bond orders in phosphine oxide H3P=O. You may want to use the Multiwfn package. 
